# CII proyect: Street Data Preparation For Network Analysis
### Process:
- Create street layer    
***Run topology of streets in ArcGIS***
- Create bus line buffer
- calculate travel time   
***Run Network Analyst in ArcGIS***

In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
# import local goverments
df_muni = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/municipio/municipio.shp")
df_muni = df_muni.loc[df_muni.nam == 'La Matanza']

# Create street layer

In [3]:
# import streets
df_nac = gpd.read_file("../Datos/Calles/RedVialNacional.geojson")
df_pro = gpd.read_file("../Datos/Calles/RedVialProvincial.geojson")
df_urb = gpd.read_file("../Datos/Calles/RedVialUrbana.geojson")

In [5]:
# clean street data
df_urb['hct'] = df_urb['objeto']
df_calles = pd.concat([df_nac[['hct', 'geometry']], df_pro[['hct', 'geometry']], df_urb[['hct', 'geometry']]])
df_calles.loc[df_calles.hct == 2, 'hct'] = 'Provincial'
print(df_calles.hct.unique())
print(len(df_calles))
df_calles.head()

['Nacional' 'Provincial' 'Calle']
587575


,hct,geometry
0,Nacional,"MULTILINESTRING ((-54.49869 -25.66899, -54.498..."
1,Nacional,"MULTILINESTRING ((-54.46205 -25.71356, -54.462..."
2,Nacional,"MULTILINESTRING ((-54.08179 -25.72037, -54.082..."
3,Nacional,"MULTILINESTRING ((-53.74518 -26.05548, -53.745..."
4,Nacional,"MULTILINESTRING ((-53.74290 -26.05752, -53.743..."


In [18]:
df_calles_lm = df_calles.sjoin(df_muni[['geometry']])
df_calles_lm.drop(columns='index_right', inplace=True)
print(df_calles_lm.hct.unique())
print(len(df_calles_lm))
df_calles_lm.head()

['Nacional' 'Provincial' 'Calle']
29871


,hct,geometry
887,Nacional,"MULTILINESTRING ((-58.55997 -34.68592, -58.560..."
1480,Nacional,"MULTILINESTRING ((-58.48539 -34.68688, -58.486..."
1782,Nacional,"MULTILINESTRING ((-58.52488 -34.78398, -58.524..."
2326,Nacional,"MULTILINESTRING ((-58.53002 -34.63490, -58.530..."
24174,Provincial,"MULTILINESTRING ((-58.56535 -34.64091, -58.567..."


In [19]:
# save layer
df_calles_lm.to_file("../DatosProcesados/CallesLaMatanza.shp")

### Run topology of streets in ArcGIS

# Create bus lines buffer (10m)

In [28]:
# import bus lines
df_redcol = gpd.read_file("../DatosProcesados/Urbanos/RecorridoColectivos.geojson")
# create a bus line buffer
df_redcol_buffer = df_redcol.copy() 
df_redcol_buffer = df_redcol_buffer.to_crs("EPSG:3035") 
df_redcol_buffer.geometry = df_redcol_buffer.buffer(20)
df_redcol_buffer = df_redcol_buffer.to_crs("EPSG:4326")
df_redcol_buffer.to_file("../DatosProcesados/CallesNetwork/RecorridoColectivos.shp")

# Calculate travel time
- walking time: 80m/min
- bus time: 40km/h == 670m/min

In [29]:
df_calles_lm = gpd.read_file("../DatosProcesados/CallesNetwork/Calles_topo.shp")
df_redcol_buffer = gpd.read_file("../DatosProcesados/CallesNetwork/RecorridoColectivos.shp")

In [30]:
# calculate length
df_calles_lm = df_calles_lm.to_crs("EPSG:3035")
df_calles_lm['len_m'] = df_calles_lm.length
df_calles_lm = df_calles_lm.to_crs("EPSG:4326")

In [31]:
# spatial join street with bus lines
df_redcol_buffer['bus_p'] = 1
df_calles_lm_bu = df_calles_lm.sjoin(df_redcol_buffer[['bus_p', 'geometry']], how='left', predicate='within')
df_calles_lm_bu.drop(columns='index_right', inplace=True)
df_calles_lm_bu.reset_index(inplace=True)
df_calles_lm_bu.rename(columns={df_calles_lm_bu.columns[0]:'ddup'}, inplace=True)
df_calles_lm_bu.drop_duplicates(subset='ddup', inplace=True)
df_calles_lm_bu.drop(columns='ddup', inplace=True)
df_calles_lm_bu.reset_index(inplace=True, drop=True)
df_calles_lm_bu.head()

,OBJECTID,hct,Shape_Leng,geometry,len_m,bus_p
0,1,Nacional,0.268575,"LINESTRING (-58.70101 -34.90636, -58.70161 -34...",25442.975608,NaN
1,2,Nacional,0.082431,"LINESTRING (-58.51885 -34.73520, -58.51885 -34...",7863.593912,NaN
2,3,Nacional,0.003143,"LINESTRING (-58.48866 -34.69103, -58.48859 -34...",198.774530,1.0
3,4,Nacional,0.040386,"LINESTRING (-58.52835 -34.65379, -58.52825 -34...",6544.503996,NaN
4,5,Provincial,0.030618,"LINESTRING (-58.59084 -34.64575, -58.59089 -34...",3512.694100,NaN


In [32]:
# calculate travel time for each street
df_calles_lm_bu['ttime_walking'] = df_calles_lm_bu['len_m'] / 80
df_calles_lm_bu['ttime_wbus'] = df_calles_lm_bu['len_m'] / 80
df_calles_lm_bu.loc[df_calles_lm_bu.bus_p.notnull(), 'ttime_wbus'] = df_calles_lm_bu['len_m'] / 670
df_calles_lm_bu.head()

,OBJECTID,hct,Shape_Leng,geometry,len_m,bus_p,ttime_walking,ttime_wbus
0,1,Nacional,0.268575,"LINESTRING (-58.70101 -34.90636, -58.70161 -34...",25442.975608,NaN,318.037195,318.037195
1,2,Nacional,0.082431,"LINESTRING (-58.51885 -34.73520, -58.51885 -34...",7863.593912,NaN,98.294924,98.294924
2,3,Nacional,0.003143,"LINESTRING (-58.48866 -34.69103, -58.48859 -34...",198.774530,1.0,2.484682,0.296678
3,4,Nacional,0.040386,"LINESTRING (-58.52835 -34.65379, -58.52825 -34...",6544.503996,NaN,81.806300,81.806300
4,5,Provincial,0.030618,"LINESTRING (-58.59084 -34.64575, -58.59089 -34...",3512.694100,NaN,43.908676,43.908676


In [33]:
# save result
df_calles_lm_bu.to_file("../DatosProcesados/CallesNetwork/Calles_traveltime.shp")

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  


### Run Network Analyst in ArcGIS